# Main Ingredients

Now that you have learned many of the fundamental procedures executed in deep learning models, the notebooks that follow will provide various examples using the Python API. These notebooks are designed to closely mirror the complementary R notebooks.

In [ ]:
# utility packages
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [ ]:

# modeling packages
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_blobs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import utils

## Simple Linear Regression

In [ ]:
n = 1000   # n observations
b = 30     # intercept
a = 5      # slope

In [ ]:
random.seed(123)
X = [random.uniform(-1, 1) for ob in range(n)]
y = [b + a*ob + random.uniform(-1, 1) for ob in X]

df = pd.DataFrame({'X': X, 'y': y})
df.head()

In [ ]:
df.plot.scatter(x='X', y='y', s=1.5)

In [ ]:
X = df[['X']]
y = df['y']

reg = LinearRegression().fit(X, y)

print(f'''
estimated intercept = {reg.coef_}
estimated slope = {reg.intercept_}
loss score = {mean_squared_error(y, reg.predict(X))}
''')

## Simple regression with a perceptron

In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(units=1, input_dim=1))

In [ ]:
# compile the keras model
model.compile(loss='mse', optimizer='sgd')

In [ ]:
# Fit the model
model.fit(X, y, batch_size=32, epochs=15)

In [ ]:
model.get_weights()

## Binary Classification

In [ ]:
X, y = make_blobs(n_samples=1000, centers=2, n_features=1)

In [ ]:
pd.DataFrame({'X': np.squeeze(X), 'y': y}).head()

In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(units=1, input_dim=1, activation = "sigmoid"))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='sgd')

# fit the model
history = model.fit(X, y, epochs=50, verbose=False)

In [ ]:
model.evaluate(X, y)

In [ ]:
pd.DataFrame({
    'epoch': history.epoch,
    'loss': history.history['loss']
}).plot.scatter(x='epoch', y='loss')

## Learning rate and momentum

In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(units=1, input_dim=1, activation = "sigmoid"))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.2))

# fit the model
model.fit(X, y, epochs=10)

In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(units=1, input_dim=1, activation = "sigmoid"))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.01, momentum=0.5))

# fit the model
model.fit(X, y, epochs=10)

## Non-linear Patterns

In [ ]:
df = pd.DataFrame({
    'X': np.arange(0, 6, 0.01),
    'y': np.sin(np.arange(0, 6, 0.01))
})

# add random noise to response variable
e = np.random.uniform(-0.2, 0.2, (df.shape[0], 1))
df[['y']] = df[['y']] + e

# plot
df.plot.scatter(x='X', y='y', s=1.5)

In [ ]:
X = df[['X']]
y = df['y']

In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(units=16, input_dim=1, activation = "relu"))
model.add(Dense(units=1))

# compile the keras model
model.compile(loss='mse', optimizer=SGD(learning_rate=0.01, momentum=0.9))

# fit the model
model.fit(X, y, batch_size=32, epochs=50, verbose=False)

In [ ]:
df[['pred']] = model.predict(X)
# plot
df.plot.scatter(x='X', y='y', s=1.5)
plt.plot(df[['X']], df[['pred']], c='red')

In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(units=16, input_dim=1, activation="relu"))
model.add(Dense(units=16, activation="relu"))
model.add(Dense(units=1))

# compile the keras model
model.compile(loss='mse', optimizer=SGD(learning_rate=0.01, momentum=0.9))

# fit the model
model.fit(X, y, batch_size=16, epochs=50, verbose=False)
          
# plot results
df[['pred']] = model.predict(X)
# plot
df.plot.scatter(x='X', y='y', s=1.5)
plt.plot(df[['X']], df[['pred']], c='red')

## Multi-predictor Multi-class Classification

In [ ]:
X, y = make_blobs(n_samples=2000, centers=4, n_features=3)

In [ ]:
# categorical encoding of y array
y = utils.to_categorical(y)
y

In [ ]:
X.shape[1]

In [ ]:
y.shape

### Fit model using validation

In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(units=16, input_dim=X.shape[1], activation="relu"))
model.add(Dense(units=y.shape[1], activation='softmax'))

# compile the keras model
model.compile(
    loss='categorical_crossentropy', 
    optimizer=SGD(learning_rate=0.01, momentum=0.9),
    metrics='accuracy'
)

# fit the model
history = model.fit(X, y, batch_size=32, epochs=20, validation_split=0.2, verbose=False)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(10,6))
plt.show()